## XGBoost
- XGBoost : Extreme Gradient Boosting 의 약자
   - "Greedy Function Approximation: A Gradient Boosting Machine", by Friedman. 논문에서 유래됨
   - Gradient Boost Tree에 기반한다.
   - 지도학습의 수학적 원리에 기반한다. : y_i의 예측값은 j번째 가중치와 xij 곱의 합
   - objective function(목적함수) = training loss(훈련손실) + regularization(정규화)
   - "The task of training the model amounts to finding the best parameters 𝜃 that best fit the training data 𝑥 𝑖 and labels 𝑦 𝑖 ." : 모델을 훈련하는 작업은 훈련데이터 x_i와 라벨데이터 y_i에 가장 적합한 최상의 매개변수를 찾는 것과 같다.
   - 훈련손실은 일반적으로 평균잔차제곱이나 로지스틱 손실값을 사용한다.
   - 정규화는 모델의 복잡성을 제어하고 과적합을 방지하는데 도움을 준다.
   - 편향-분산 tradeoff에 의해 훈련손실과 정규화 값의 tradeoff가 가능하다.
   - 여러 그레디언트 부스트 tree의 결합으로 이루어진다.
   - 랜덤포레스트와 그레디언트 부스트 나무는 같다.
   - 훈련 한번에 하나의 나무 모형을 추가한다. (additive tree learning)
      - 대부분의 경우 잘 작동한다
      - 단점은 새롭게 추가되는 나무가 특정한 차원의 feature만 학습하므로 퇴화한 모형이 될 수 있다. 
   - 목적함수의 훈련손실에 t번째 예측값 식을 대입한 후 MSE 값으로 계산하여 정리한다. 테일러 확장을 적용하여 식을 정리하면 최적화 대상인 목적함수가 된다.
   - 이렇게 성립한 목적함수는 테일러 확장으로 추가한 g_i와 h_i 값에 의존하게 된다.
   
### xgboost의 모델 클래스
- 모델의 형태는 크게 regressor, classifier, ranker 3가지로 구분 된다.
   - 기본형, Dask용, PySpark용이 따로 있다. 
- classifier
   - xgboost.XGBClassifier()
   - xgboost.XGBRFClassifier()

### callback
- callback 함수를 사용하여 조기 중지, 체크 포인트 등의 기능을 사용할 수 있다.
   - xgb.callback.EvaluationMoinitor()
   - xgb.callback.EarlyStopping()
   - xgb.callback.LearningRateScheduler()
   - xgb.callback.TrainingCheckPoint()
- xgb.train()의 파라미터로 callbacks=[]에서 설정할 수 있다.
   - early_stop = xgb.callback.EarlyStopping()
   - xgb.train(callbacks=[early_stop])
- 각 기능별로 커스텀도 가능하다.   
- callback docs
   - https://xgboost.readthedocs.io/en/stable/python/callbacks.html
   - https://xgboost.readthedocs.io/en/stable/python/examples/callbacks.html#sphx-glr-python-examples-callbacks-py

### DASK
- python 기반으로 구축된 병렬 컴퓨팅 라이브러리
- DASK 를 사용하면 분산된 작업자를 쉽게 관리할 수 있다. 
- 대규모 분산 데이터 과학 워크플로를 처리하는데 탁월하다.
- https://xgboost.readthedocs.io/en/stable/tutorials/dask.html
- 기존 class들과 같은 Dask 용 class 들이 따로 있음
   - DaskDMatrix()
   - DaskXGBClassifier()
   - DaskXGBRFClassifier()
   - 등등
   
### PySpark
- PySpark 용 XGBoost classifier
   - xgb.spark.SparkXGBClassifier()

### parameter
- Global configuration : xgboost.config_context()로 설정가능**
    - verbosity : 훈련 과정에 대한 메시지 출력여부, 
       - 0 : silent, 1 :  warning, 2 : info, 3 : debug
    - use_rmm : RAPIDS Memory Manager의 사용하여 GPU 메모리 할당 여부
       - True, False
       - RMM 플러그인을 활성화하여 XGBoost를 빌드하는 경우 적용 가능
- General parameters
    - booster : 부스터 종류 설정, default=gbtree
       - gbtree, dart, gblinear
    - device : XGBoost 실행을 위한 장치 설정
       - cpu : use cpu
       - cuda : use gpu
       - gpu
    - verbosity
    - validate_parameters : 입력 매개변수 유효성 검사, 알수 없는 매개변수가 있으면 경고
    - nthread : 병렬 스레드의 수
- parameter for tree booster    
    - eta : 과적합 방지를 위한 step size shrinkage, default=0.3
    - gamma : tree의 리프 노드에 추가 파티션을 만드는데 필요한 최소 손실 감소, 감마가 클 수록 알고리즘은 더 보수적이게 된다.
    - max_depth : tree의 최대 깊이, depth가 커질 수록 더 복잡해지고 과적합이 커진다. XGBoost는 deep tree를 훈련할 때 메모리를 많이 사용한다. default=6
       - 0 : 깊이에 제한이 없음
    - tree_method : 트리 구성 알고리즘, 
       - auto : hist와 같음
       - exact : 탐욕 알고리즘, 모든 분할 후보를 열거한다.
       - approx : 분위수 근사치 + 그래디언트 히스토그램을 이용한 그리디 알고리즘의 근사화
       - hist : 더 빠른 히스토그램 최적화 근사 탐욕 알고리즘
    - process_type : 부스팅 타입 설정
       - default : 새로운 tree를 만드는 일반적인 boosting
       - update : 초기 모델의 트리를 가져와 지정된 업데이트 시퀀스가 실행되어 수정된 트리가 새모델에 추가됨
       - refresh, prune 내장 업데이트 프로그램 사용가능
    - grow_policy : 새노드가 트리에 추가되는 방식을 제어함, default=depthwise
       - tree_method가 hist 또는 about으로 설정된 경우에만 사용가능
       - depthwise : 루트노드에서 가장 가까운 노드에서 분할
       - lossguide : 손실 변화가 가장 큰 노드에서 분할
    - max_leaves : 추가할 최대 노드의 수, 정확한 트리 방버에서는 사용되지 않는다.
    - max_bin : 개별 bin의 최대치, max_bin이 커지면 계산시간이 늘어나지만 분할의 최적성이 향상된다.
    - interaction_constraints : 상호작용에 대한 제약 조건, 이중 리스트 형식으로 설정함, 
- parameter for categorical feature    
    - 카테고리 데이터의 훈련과정에서 사용됨, tree_method=exact 에서는 작동이 안된다.
    - max_cat_to_onehot : 범주형 데이터에 대해 원한인코딩 기반 분할을 사용할지 여부 설정, 범주수가 임계값보다 작으면 원핫 인코딩 선택, 임계값과 같거나 크면 범주가 하위 노드로 분할 된다.
    - max_cat_threshold : 각 분할에 사용되는 최대 범주의 수, 파티션 기반 분할에서만 사용할 수있다

- boster=dart, booster=gblinear
    - booster=dart, booster=gblinear 각각에 대한 parameter를 따로 설정할 수 있다.
- learning task parameters    
    - 학습과제와 해당학습 목표를 지정한다. 객관적인 옵션
    - metrics 설정
    - objective : default=reg:squarederror
       - reg, binary, count, survival, multi, rank 등 옵션에 따른 지표를 설정할 수 있다.
    - eval_metric : 검증 데이터에 대한 평가 메트릭
       - 회귀 : rmse , 분류 : logloss, rank:map : 평균정밀도(mean average precision)
       - 회귀 : rmse, rmsel, mae, mape, 
       - 분류 : logloss, auc, aucpr
- command line parameters
    - num_rounds : 부스팅의 라운드 수
    - data : path of training data
    - task : 
       - train : 데이터를 사용한 훈련
       - pred : test:data를 사용한 예측값 생성
       - eval : eval[name]=filename 으로 지정된 통계를 평가함
       - dump : 학습된 모델을 텍스트 형식으로 덤프 (저장, 내보내기)
- 더 많은 특징

** control overfitting **
- 훈련 정확도는 높은데 테스트 정확도가 낮으면 과적합이 된 것
- 과적합을 조정하는 방법
   - 모델의 복잡성을 직접 제어한다. : max_depth, min_child_weight, gamma
   - 무작위성을 높여서 noise에 강하도록 만든다. : subsample, colsample_bytree
   - eta 값을 감소시키고 num_round를 증가시킨다.
- tree methods 
   - 강화된 트리 모델을 훈련하기 위해 알고리즘을 선택하는데 사용할 수 있는 파라미터
   - updater : refresh, prune, sync 
   - tree_method : exact, approx, hist

** Model IO의 JSON 직렬화 저장/로드 **
- XGBoost에서 사용되는 JSON 직렬화 방법
   - XGBoost 1.0.0에서는 훈련을 위한 모델과 하이퍼 파라미터의 저장 및 로드를 위해서 JSON 사용을 도입했음
   - booster.save_model("model.json")
- XGBoost의 특징
   - (1) XGBoost는 트리 모델에 초점을 맞춘 그래디언트 부스팅 라이브러리이다.
   - (2) XGBoost 내부에는 2가지 다른 부분이 있다.
      - 나무로 구성된 모델
      - 모델 구축에 사용되는 하이퍼 파라미터 및 구성
- booster.save_model() 을 호출하면 XGBoost는 trees, model parameters, objective function을 저장한다.
   - prediction, evaluation을 위해서 다른 사람과 공유하고 다른 하이퍼 파라미터 세트를 사용하여 훈련을 계속할 수 있다.
- memory snapshot : 메모리 스냅샷
   - 메모리 기반 직렬화 방법
   - 이러한 과정에서 다양한 방식으로 모델을 공유하고 훈련을 지속하게 되는데 JSON 직렬화 출력을 통해서 이전 훈련을 계속할 수 있도록 충분한 정보를 제공할 수 있다.
   - python에서 pickle 모듈을 통해서 pickled 될 떄 사용 된다.
   - R, JVM 등에서 저장 및 로드 할때 사용된다.
- JSON 직렬화 방법
   - 확장자를 .json 으로 저장한다.ㄹ
   - python : bst.save_model("model_file_name.json") 
- 버전간 호환성
   - save_model()을 사용하면 trees와 objective로 구성된 모델은 이전버전과 이후버전에서 액세스할 수 있다. 
   - pickle.dump()로 저장하는 경우 이후 버전의 XGBoost에서 해당 모델을 액세스하지 못 할 수 있다. pickle은 안정적인 직렬화 형식이 아니다.
   - pickle.dump()로 저장한 모델을 특정 버전의 XGBoost에서 로드하고 다시 save_model()로 저장하면 호환문제를 해결할 수 있다. 
- Python에서 저장, 로드
   - bst = xgboost.train()
   - config = bst.save_config() ---> save
   - print(config) ---> json 형식으로 된 모델의 정보가 출력된다.
   - bst.load_config(config) ---> load
- dump_model 함수와 save_model 함수의 차이
   - XGBoost 객체는 dump_model()과 save_model() 함수가 있다. 
   - dump_model() 함수 :  text, json, dot 등 읽을 수 있는 형식으로 내보낼 수 있다. 이것은 모델을 시각화하기 위한 방법이다. XGBoost 로 다시 로드하면 안된다.
   - save_model() 함수는 XGBoost 출력모델을 쉽게 사용할 있는 문서화 된 schema를 포함하고 있다.

** DART 부스터 **
- booster="dart"
   - XGBoost는 작은 학습률과 수많은 회귀 트리를 결합한다. 일찍 추가된 트리가 더 중요하고 나중에 추가된 트리는 중요하지 않게 된다.
   - dropout 기술을 트리에 적용하여 특정 상황에서 결과가 더 나아짐 : Vinayak and Gilad-Bachrach
   - 뉴부스터 "dart"
   - https://proceedings.mlr.press/v38/korlakaivinayak15.pdf
- feature의 dropout
   - 과적합을 방지하는 효과가 있다.
   - 무작위 dropout으로 인해 훈련속도가 gbtree보다 느리거나 조기 종료가 안정적이지 않을 수 있다. 
   - drop 된 나무의 leaf score의 합과 새로운 tree의 leaf score의 합의 관계를 통해서 목적함수를 만들 수 있다. 
- parameters
   - gbtree 부스터를 상속받는다. : eta, gamma, max_depth 등의 매개변수
   - 추가 매개변수 : 
      - sample_type : 샘플링 알고리즘의 타입
         - uniform : uniformly type으로 drop할 tree 선택
         - weighted : 가중치에 따라서 drop tree 선택
      - nomalized_type : 일반적인 알고리즘의 type
         - tree : drop 된 tree와 같은 가중치를 가진 new tree
         - forest : drop 된 trees들의 가중치의 합을 가진 new tree
      - rate_drop : dropout 비율
      - skip_drop : dropout을 건너뛸 확률

### kaggle higs competition use xgboost blog
- XGBoost는 그래디언트 부스팅의 병렬 구현
   - 병렬적이고, 빠르고, 효율적이고, 조정 가능한 매개변수가 제공된다.
- 매개변수 튜닝
   - small shrinkage eta value 값을 조정하면 좋은 학습결과를 도출할 수 있다.
   - eta = 0.01(small shrinkage)
   - max_depth = 9
   - sub_sample = 0.9 : 과적합 방지를 위한 임의성(randomness) 부여
   - num_rounds = 3000 (shrinkage 값이 작은 경우 많은 나무가 필요하다.)
   - 다른 매개변수는 그대로 둠
   - 다른 상황에서는 적합하지 않은 선택일 수 있음
- 매개변수 튜닝에 따른 모델링 결과
   - shrinkage가 너무 느리고, 훈련시간이 길어짐, 교차 검증(cross validate)에 더 오랜 시간이 걸림.
   - AMS score에 대한 수학적 의미
- feature engineering이 핵심
   - 일반적인 아이디어에서 출발 : 신호와 배경 샘플에 대한 과학적 이해를 바탕으로 feature engineering??
- feature selection
   - feature selection을 위해서 어떤 논리적 접근, 도메인 지식이 사용되거나 자동기능선택(예를들면 PCA)이 있다.
   - competition의 score의 특성에 따라서 자동기능선택 방법이 안 좋을 수 있다. -> 중요한 feature를 삭제할 수 있기때문.
- 교훈
   - 프로세스의 자동화 : 스크립트를 사용하여 CV, training+testing, parameter scan 등을 사용하면 시간을 절약할 수 있다. 

### xgboost 모델 저장, 로드
- 모델 훈련 
   - bst = xgb.train(params, dtrain, num_round)
- 모델 저장 : json 파일 저장 추천
   - json 파일로 저장 : bst.save_model("./build_model/xgb_best.json")
   - pickle 파일로 저장 : pickle.dumps(bst, oepn(""./build_model/xgb_best.pkl", "wb"))
- 모델 로드
   - json 로드 : bst2 = xgb.Booster("./build_model/xgb_best.json")
   - pickle 로드 : bst2 = pickle.load(open("./build_model/xgb_best.pkl", "rb"))
- json 저장 -> 로드 -> 다시 pickle 로 저장 해도 됨
       

In [1]:
!conda list "^xgb"

# packages in environment at C:\DS\Anaconda3\envs\dev_env:
#
# Name                    Version                   Build  Channel
xgboost                   2.0.3                    pypi_0    pypi


In [2]:
from sklearn import datasets

In [5]:
X, y = datasets.fetch_california_housing(return_X_y=True, as_frame=True)
X.shape, y.shape

((20640, 8), (20640,))

In [9]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [12]:
np.unique(y)

array([0.14999, 0.175  , 0.225  , ..., 4.991  , 5.     , 5.00001])

In [48]:
import multiprocessing
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

In [49]:
xgb_model = xgb.XGBRegressor(
    n_jobs=multiprocessing.cpu_count() // 2, tree_method="hist"
)
clf = GridSearchCV(
    xgb_model, {"max_depth": [2, 4, 6], "n_estimators": [50, 100, 200]}
)
clf.fit(X, y)
print(clf.best_score_)
print(clf.best_params_)

0.688555675183378
{'max_depth': 4, 'n_estimators': 50}


In [50]:
from sklearn.preprocessing import StandardScaler

In [54]:
ss = StandardScaler()
ss_X = ss.fit_transform(X)
ss_X = pd.DataFrame(ss_X, columns=X.columns)
ss_X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.344766,0.982143,0.628559,-0.153758,-0.974429,-0.049597,1.052548,-1.327835
1,2.332238,-0.607019,0.327041,-0.263336,0.861439,-0.092512,1.043185,-1.322844
2,1.782699,1.856182,1.155620,-0.049016,-0.820777,-0.025843,1.038503,-1.332827
3,0.932968,1.856182,0.156966,-0.049833,-0.766028,-0.050329,1.038503,-1.337818
4,-0.012881,1.856182,0.344711,-0.032906,-0.759847,-0.085616,1.038503,-1.337818
...,...,...,...,...,...,...,...,...
20635,-1.216128,-0.289187,-0.155023,0.077354,-0.512592,-0.049110,1.801647,-0.758826
20636,-0.691593,-0.845393,0.276881,0.462365,-0.944405,0.005021,1.806329,-0.818722
20637,-1.142593,-0.924851,-0.090318,0.049414,-0.369537,-0.071735,1.778237,-0.823713
20638,-1.054583,-0.845393,-0.040211,0.158778,-0.604429,-0.091225,1.778237,-0.873626


In [56]:
ss_X.describe().loc[["std", "mean"]]

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00
mean,6.609700e-17,5.508083e-18,6.609700e-17,-1.060306e-16,-1.101617e-17,3.442552e-18,-1.079584e-15,-8.526513e-15


In [57]:
clf.fit(ss_X, y)
print(clf.best_score_)
print(clf.best_params_)

0.688555675183378
{'max_depth': 4, 'n_estimators': 50}


In [59]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16512, 8), (4128, 8), (16512,), (4128,))

In [60]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [61]:
dtrain.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [62]:
dtest.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [63]:
param = {"max_depth": 2, "eta": 1, "objective": "reg:squaredlogerror"}
watchlist = [(dtest, "eval"), (dtrain, "train")]
num_round = 2

bst = xgb.train(param, dtrain, num_boost_round=num_round, evals=watchlist)
bst

[0]	eval-rmsle:0.45143	train-rmsle:0.45403
[1]	eval-rmsle:0.30861	train-rmsle:0.30585


In [66]:
pred = bst.predict(dtest)
labels = dtest.get_label()

print(f"error = { sum(1 for i in range(len(pred)) if int(pred[i] > 0.5) != labels[i]) / float(len(pred)) }")

error = 0.9978197674418605


In [79]:
xgb.get_config()

{'use_rmm': False, 'verbosity': 1}

### XGBoost : dart with titanic

In [3]:
!ls ../../../sample_data/titanic

gender_submission.csv
test.csv
train.csv


In [13]:
train_data = pd.read_csv("../../../sample_data/titanic/train.csv")
test_data = pd.read_csv("../../../sample_data/titanic/test.csv")
data = pd.concat([train_data, test_data], axis=0).reset_index()

data.head(2)

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [7]:
def find_title(name) : 
    
    '''
    Name의 데이터 샘플에서 특수문자를 제거하고 title을 추출하여 반환한다.
    '''
    remove_char = ',./:;?!#@"'
    for rc in remove_char : 
        name = name.replace(rc, "")
    
    title_list = ['Mr', 'Mrs', 'Miss', 'Ms', 'Master', 
                  'Dr', 'Don', 'Dona', 'Sir', 'Rev', 'Mme', 
                  'Major', 'Lady', 'Mlle', 'Capt', 'Countess', 'Jonkheer', 'Col']
    result = []
    for name in name.split(" ") : 
        if name in title_list : 
            result.append(name)
    
    return result


def df_preprocessor(df) : 
    
    '''
    feature preprocessing
    '''
    df["Age"] = df["Age"].fillna(df["Age"].mean().round(0))
    age_bins = [0, 1, 12, 18, 64, 100]
    age_labels = ["toddler", "children", "teenager", "adult", "old"]
    df["generation"] = pd.cut(df["Age"], bins=age_bins, labels=age_labels)
    
    df["title_name"] = df["Name"].apply(lambda x: find_title(x)[0])
    df.loc[df["title_name"].isin(["Mlle", "Mme", "Ms", "Dona"]), "title_name"] = "Mrs"
    df.loc[df["title_name"].isin(["Lady"]), "title_name"] = "Miss"
    df.loc[df["title_name"].isin(["Col", "Capt"]), "title_name"] = "army"
    df.loc[df["title_name"].isin(["Don", "Sir", "Countess", "Jonkheer"]), "title_name"] = "peerage"

    df.drop(["Cabin"], axis=1, inplace=True)
    df["Embarked"] = df["Embarked"].fillna("S")
    
    df["Fare"] = df["Fare"].fillna(df["Fare"].mean().round(0))
    df.set_index("index", inplace=True)
    
    return df

In [14]:
clean_data = df_preprocessor(data)
clean_data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,generation,title_name
index,,,,,,,,,,,,,
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,adult,Mr
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,adult,Mrs


In [15]:
train_data = clean_data.iloc[:891]
test_data = clean_data.iloc[891:]

X = train_data.drop(["PassengerId", "Survived", "Name", "Ticket"], axis=1)
y = train_data["Survived"]

X.shape, y.shape

((891, 9), (891,))

In [16]:
num_features = ["Age", "Fare", "SibSp", "Parch"]
cat_features = list(set(X.columns).difference(num_features))

X[cat_features] = X[cat_features].astype("category")
X[num_features] = X[num_features].astype(int)
X.dtypes

Pclass        category
Sex           category
Age              int32
SibSp            int32
Parch            int32
Fare             int32
Embarked      category
generation    category
title_name    category
dtype: object

In [34]:
X_onehot = pd.get_dummies(X)
X_onehot.head(2)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,...,generation_old,title_name_Dr,title_name_Major,title_name_Master,title_name_Miss,title_name_Mr,title_name_Mrs,title_name_Rev,title_name_army,title_name_peerage
index,,,,,,,,,,,,,,,,,,,,,
0,22,1,0,7,False,False,True,False,True,False,...,False,False,False,False,False,True,False,False,False,False
1,38,1,0,71,True,False,False,True,False,True,...,False,False,False,False,False,False,True,False,False,False


In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
enc = LabelEncoder()
y_enc = enc.fit_transform(y)
y_enc

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [19]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_onehot, 
                                                    y_enc, 
                                                    test_size=0.2, 
                                                    random_state=50)
X_train.shape, y_train.shape

((712, 26), (712,))

In [36]:
dtrain = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [43]:
dtrain.num_row(), dtrain.num_col()

(712, 26)

In [44]:
dtest.num_row(), dtest.num_col()

(179, 26)

In [128]:
dart_param = {
    "booster": "dart",
    "device": "cuda",
    "eta": 0.1,
    "gamma": 0.3, 
    "max_depth": 10,
    "learing_rate": 0.1,
    "objective": "binary:logistic",
    "sample_type": "uniform",
    "normalize_type": "tree",
    "rate_drop": 0.1, 
    "skip_drop": 0.5,
    "eval_metrics": "logloss",
    "verbosity": 0,
}
num_round = 800
watchlist = [(dtest, "eval"), (dtrain, "train")]

boosting = xgb.train(
    params=dart_param, 
    dtrain=dtrain, 
    num_boost_round=num_round,
    evals=watchlist,
    early_stopping_rounds=500
)

boosting

[0]	eval-logloss:0.64620	train-logloss:0.60803
[1]	eval-logloss:0.60875	train-logloss:0.56692
[2]	eval-logloss:0.57781	train-logloss:0.53185
[3]	eval-logloss:0.55276	train-logloss:0.50241
[4]	eval-logloss:0.53155	train-logloss:0.47691
[5]	eval-logloss:0.52361	train-logloss:0.46727
[6]	eval-logloss:0.50682	train-logloss:0.44566
[7]	eval-logloss:0.49252	train-logloss:0.42725
[8]	eval-logloss:0.48093	train-logloss:0.41156
[9]	eval-logloss:0.47056	train-logloss:0.39801
[10]	eval-logloss:0.46328	train-logloss:0.38720
[11]	eval-logloss:0.45659	train-logloss:0.37555
[12]	eval-logloss:0.45093	train-logloss:0.36402
[13]	eval-logloss:0.44718	train-logloss:0.35408
[14]	eval-logloss:0.44431	train-logloss:0.34526
[15]	eval-logloss:0.44095	train-logloss:0.33757
[16]	eval-logloss:0.44040	train-logloss:0.33670
[17]	eval-logloss:0.44032	train-logloss:0.33487
[18]	eval-logloss:0.43893	train-logloss:0.32788
[19]	eval-logloss:0.43902	train-logloss:0.32066
[20]	eval-logloss:0.43895	train-logloss:0.31696
[2

[170]	eval-logloss:0.45713	train-logloss:0.23289
[171]	eval-logloss:0.45710	train-logloss:0.23290
[172]	eval-logloss:0.45712	train-logloss:0.23284
[173]	eval-logloss:0.45718	train-logloss:0.23282
[174]	eval-logloss:0.45723	train-logloss:0.23277
[175]	eval-logloss:0.45727	train-logloss:0.23277
[176]	eval-logloss:0.45726	train-logloss:0.23272
[177]	eval-logloss:0.45726	train-logloss:0.23272
[178]	eval-logloss:0.45726	train-logloss:0.23272
[179]	eval-logloss:0.45726	train-logloss:0.23272
[180]	eval-logloss:0.45726	train-logloss:0.23272
[181]	eval-logloss:0.45725	train-logloss:0.23272
[182]	eval-logloss:0.45726	train-logloss:0.23273
[183]	eval-logloss:0.45726	train-logloss:0.23273
[184]	eval-logloss:0.45726	train-logloss:0.23273
[185]	eval-logloss:0.45730	train-logloss:0.23271
[186]	eval-logloss:0.45730	train-logloss:0.23271
[187]	eval-logloss:0.45730	train-logloss:0.23271
[188]	eval-logloss:0.45730	train-logloss:0.23271
[189]	eval-logloss:0.45729	train-logloss:0.23271
[190]	eval-logloss:0

[338]	eval-logloss:0.45774	train-logloss:0.23240
[339]	eval-logloss:0.45775	train-logloss:0.23240
[340]	eval-logloss:0.45775	train-logloss:0.23240
[341]	eval-logloss:0.45775	train-logloss:0.23240
[342]	eval-logloss:0.45777	train-logloss:0.23240
[343]	eval-logloss:0.45777	train-logloss:0.23240
[344]	eval-logloss:0.45777	train-logloss:0.23240
[345]	eval-logloss:0.45779	train-logloss:0.23238
[346]	eval-logloss:0.45779	train-logloss:0.23238
[347]	eval-logloss:0.45779	train-logloss:0.23239
[348]	eval-logloss:0.45776	train-logloss:0.23238
[349]	eval-logloss:0.45776	train-logloss:0.23238
[350]	eval-logloss:0.45776	train-logloss:0.23238
[351]	eval-logloss:0.45776	train-logloss:0.23238
[352]	eval-logloss:0.45777	train-logloss:0.23235
[353]	eval-logloss:0.45778	train-logloss:0.23235
[354]	eval-logloss:0.45779	train-logloss:0.23234
[355]	eval-logloss:0.45779	train-logloss:0.23234
[356]	eval-logloss:0.45778	train-logloss:0.23234
[357]	eval-logloss:0.45778	train-logloss:0.23234
[358]	eval-logloss:0

[506]	eval-logloss:0.45814	train-logloss:0.23208
[507]	eval-logloss:0.45814	train-logloss:0.23208
[508]	eval-logloss:0.45814	train-logloss:0.23208
[509]	eval-logloss:0.45816	train-logloss:0.23208
[510]	eval-logloss:0.45816	train-logloss:0.23208
[511]	eval-logloss:0.45816	train-logloss:0.23208
[512]	eval-logloss:0.45817	train-logloss:0.23208
[513]	eval-logloss:0.45817	train-logloss:0.23208
[514]	eval-logloss:0.45818	train-logloss:0.23208
[515]	eval-logloss:0.45818	train-logloss:0.23208
[516]	eval-logloss:0.45818	train-logloss:0.23208
[517]	eval-logloss:0.45818	train-logloss:0.23208
[518]	eval-logloss:0.45819	train-logloss:0.23208
[519]	eval-logloss:0.45819	train-logloss:0.23207
[520]	eval-logloss:0.45819	train-logloss:0.23209
[521]	eval-logloss:0.45819	train-logloss:0.23210
[522]	eval-logloss:0.45819	train-logloss:0.23210
[523]	eval-logloss:0.45819	train-logloss:0.23210
[524]	eval-logloss:0.45822	train-logloss:0.23208
[525]	eval-logloss:0.45824	train-logloss:0.23208
[526]	eval-logloss:0

[674]	eval-logloss:0.45832	train-logloss:0.23190
[675]	eval-logloss:0.45834	train-logloss:0.23189
[676]	eval-logloss:0.45833	train-logloss:0.23187
[677]	eval-logloss:0.45832	train-logloss:0.23188
[678]	eval-logloss:0.45832	train-logloss:0.23188
[679]	eval-logloss:0.45832	train-logloss:0.23188
[680]	eval-logloss:0.45832	train-logloss:0.23188
[681]	eval-logloss:0.45832	train-logloss:0.23188
[682]	eval-logloss:0.45832	train-logloss:0.23188
[683]	eval-logloss:0.45832	train-logloss:0.23189
[684]	eval-logloss:0.45832	train-logloss:0.23189
[685]	eval-logloss:0.45832	train-logloss:0.23189
[686]	eval-logloss:0.45833	train-logloss:0.23188
[687]	eval-logloss:0.45833	train-logloss:0.23188
[688]	eval-logloss:0.45833	train-logloss:0.23188
[689]	eval-logloss:0.45833	train-logloss:0.23188
[690]	eval-logloss:0.45835	train-logloss:0.23188
[691]	eval-logloss:0.45837	train-logloss:0.23188
[692]	eval-logloss:0.45838	train-logloss:0.23187
[693]	eval-logloss:0.45838	train-logloss:0.23187
[694]	eval-logloss:0

In [132]:
boosting.best_score

0.23174531639585

In [133]:
boosting.attributes()

{'best_iteration': '799', 'best_score': '0.23174531639585'}

In [134]:
preds = boosting.predict(dtest)
labels = dtest.get_label()

print(f"""
~~~ correct rate ~~~
{sum(1 for i in range(len(preds)) if int(preds[i] > 0.5) == labels[i]) 
    / float(len(preds)) : .3f}

~~~ error rate ~~~
{sum(1 for i in range(len(preds)) if int(preds[i] > 0.5) != labels[i]) 
    / float(len(preds)) : .3f}    
""")


~~~ correct rate ~~~
 0.832

~~~ error rate ~~~
 0.168    



In [150]:
boosting.save_model("model_1.json")
boosting2 = xgb.Booster(model_file="model_1.json")
preds2 = boosting2.predict(dtest)

assert np.sum(np.abs(preds2 - preds)).round(5) == 0

In [154]:
config = boosting.save_config()
print(config)

{"learner":{"generic_param":{"device":"cuda:0","fail_on_invalid_gpu_id":"0","n_jobs":"0","nthread":"0","random_state":"0","seed":"0","seed_per_iteration":"0","validate_parameters":"1"},"gradient_booster":{"dart_train_param":{"normalize_type":"tree","one_drop":"0","rate_drop":"0.100000001","sample_type":"uniform","skip_drop":"0.5"},"gbtree":{"gbtree_model_param":{"num_parallel_tree":"1","num_trees":"800"},"gbtree_train_param":{"process_type":"default","tree_method":"auto","updater":"grow_gpu_hist","updater_seq":"grow_gpu_hist"},"name":"gbtree","specified_updater":false,"tree_train_param":{"alpha":"0","cache_opt":"1","colsample_bylevel":"1","colsample_bynode":"1","colsample_bytree":"1","eta":"0.100000001","gamma":"0.300000012","grow_policy":"depthwise","interaction_constraints":"","lambda":"1","learning_rate":"0.100000001","max_bin":"256","max_cat_threshold":"64","max_cat_to_onehot":"4","max_delta_step":"0","max_depth":"10","max_leaves":"0","min_child_weight":"1","min_split_loss":"0.3000

### XGBoost : CV

In [174]:
dart_param = {
    "booster": "dart",
    "device": "cuda",
    "eta": 0.011,
    "gamma": 0.01, 
    "max_depth": 6,
    "learing_rate": 0.1,
    "objective": "binary:logistic",
    "sample_type": "uniform",
    "normalize_type": "tree",
    "rate_drop": 0.1, 
    "skip_drop": 0.5,
    "eval_metrics": "logloss",
    "verbosity": 0,
}
num_round = 2
watchlist = [(dtest, "eval"), (dtrain, "train")]

print("running cv")
xgb.cv(
    dart_param, 
    dtrain, 
    num_round,
    nfold=5,
    metrics={"error"},
    seed=0,
    callbacks=[xgb.callback.EvaluationMonitor(show_stdv=True)]
)


running cv
[0]	train-error:0.37079+0.00870	test-error:0.37090+0.03476
[1]	train-error:0.37079+0.00870	test-error:0.37090+0.03476


,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.370793,0.008704,0.370895,0.034758
1,0.370793,0.008704,0.370895,0.034758


In [173]:
print("running cv, disable std display")
res = xgb.cv(
    dart_param, 
    dtrain, 
    num_round,
    nfold=5,
    metrics={"error"},
    seed=0,
    callbacks=[
        xgb.callback.EvaluationMonitor(show_stdv=False),
        xgb.callback.EarlyStopping(3)
    ]
)

res

running cv, disable std display
[0]	train-error:0.37079	test-error:0.37090
[1]	train-error:0.37079	test-error:0.37090


,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.370793,0.008704,0.370895,0.034758
1,0.370793,0.008704,0.370895,0.034758


### define preprocessor function 
- pos class의 가중치 계산 함수를 파라미터 값으로 설정할 수 있다. 

In [163]:
def fpreproc(dtrain, dtest, param) : 
    
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label == 1)
    param["scale_pos_weight"] = ratio
    
    return (dtrain, dtest, param)

In [170]:
float(np.sum(dtrain.get_label() == 0)) / np.sum(dtrain.get_label() == 1)

1.696969696969697

In [171]:
xgb.cv(
    dart_param,
    dtrain,
    num_boost_round=10,
    nfold=5,
    metrics={"auc"},
    seed=0,
    fpreproc=fpreproc
)

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.912749,0.009946,0.865051,0.028417
1,0.915225,0.008820,0.865903,0.027152
2,0.915383,0.008697,0.865547,0.027843
3,0.915162,0.009055,0.866504,0.028967
4,0.916588,0.010257,0.866030,0.031139
5,0.916030,0.010039,0.865819,0.031500
6,0.916144,0.010176,0.866499,0.031922
7,0.917277,0.009274,0.866702,0.031852
8,0.917784,0.009227,0.866986,0.031984
9,0.918391,0.008779,0.866563,0.032743


In [3]:
import random

In [13]:
test_df = pd.DataFrame({"x0": random.choices(population=["s", "b"], k=10)}, 
             columns=["x0"])
test_df

,x0
0,b
1,s
2,b
3,s
4,b
5,s
6,b
7,s
8,s
9,b


In [14]:
test_df2 = test_df.copy()
test_df2["x0"] = test_df["x0"].map({"s": 0, "b": 1})
test_df2

,x0
0,1
1,0
2,1
3,0
4,1
5,0
6,1
7,0
8,0
9,1


In [19]:
pd.DataFrame(np.hstack((test_df, test_df2))).rename(columns={0: "x0", 1: "x1"})

,x0,x1
0,b,1
1,s,0
2,b,1
3,s,0
4,b,1
5,s,0
6,b,1
7,s,0
8,s,0
9,b,1
